# **Notebook de transfert des donnees du site nba.com depuis le site ou un fichier intermediaire**

In [7]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import sys, os, re #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import Connexion_Transfert as ct
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import Select
from datetime import datetime
from TelechargementDonnees import JoueursSiteNba, CreationDriverFirefox

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Données de joueurs 
Il s'agit de transfere dans le cas des donnees de joueuer telechargees en amont, ou d'ajouter des joueurs non presents au moment d'insertion de match

>## 1.1 Enumerations des types de position
A creer a la main

In [10]:
#creation de la classe
pagejoueuer=JoueursSiteNba('USA')

In [21]:
pagejoueuer.dfJoueurs.rename(columns={'Player':'nom','Position':'id_position_terrain','Height':'taille','Weight':'poids'},inplace=True)
pagejoueuer.dfJoueurs['taille']=pagejoueuer.dfJoueurs.taille.str.split('-').apply(lambda x : round((int(x[0])/3.2808)+((int(x[1])/0.39370)/100),2))
pagejoueuer.dfJoueurs['poids']=pagejoueuer.dfJoueurs.poids.str.split().apply(lambda x : round(int(x[0])/2.2046,1))
pagejoueuer.dfJoueurs['nom_simple']=pagejoueuer.dfJoueurs.nom.apply(lambda x : re.sub(' |-|\.|\'','',x).lower())

In [22]:
pagejoueuer.dfJoueurs

,nom,Team,Number,id_position_terrain,taille,poids,Last Attended,Country,nom_simple
0,PreciousAchiuwa,MIA,5,F,2.03,102.1,Memphis,Nigeria,preciousachiuwa
1,JaylenAdams,MIL,6,G,1.83,102.1,St. Bonaventure,USA,jaylenadams
2,StevenAdams,NOP,12,C,2.11,120.2,Pittsburgh,New Zealand,stevenadams
3,BamAdebayo,MIA,13,C-F,2.06,115.7,Kentucky,USA,bamadebayo
4,LaMarcusAldridge,SAS,12,C-F,2.11,113.4,Texas,USA,lamarcusaldridge
...,...,...,...,...,...,...,...,...,...
495,DelonWright,DET,55,G,1.96,83.9,Utah,USA,delonwright
496,ThaddeusYoung,CHI,21,F,2.03,106.6,Georgia Tech,USA,thaddeusyoung
497,TraeYoung,ATL,11,G,1.85,81.6,Oklahoma,USA,traeyoung
498,CodyZeller,CHA,40,F-C,2.11,108.9,Indiana,USA,codyzeller


In [ ]:
with ct.ConnexionBdd('basket','maison') as c : 
    pd.DataFrame({'id_position_terrain':pagejoueuer.dfJoueurs.id_position_terrain.unique(), 
              'nom_position_terrain':['Forward ; 3 ; ailier', 'Guard ; 1 ou 2  ; meneur ou arriere', 
                                      'Center ; 5 ; Pivot' ,'Power Forward ; 4 ; ailier Fort', 
                                      'Shooting Guard ; 2 ; arriere', 'Shooting Guard ; 2 ; arriere',
                                      'Power Forward ; 4 ; ailier Fort']}).to_sql(
        'enum_position_terrain', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

> ## 1.2 Donnees chargees depuis la page internet
Là ce n'est que le premier import, on ne gère pas la mise à jour

In [ ]:
#transfert dans la Bdd
with ct.ConnexionBdd('basket','maison') as c : 
    pagejoueuer.dfJoueurs[['nom','id_position_terrain','taille', 'poids','date_entree_nba']].to_sql(
        'joueur', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

# 2. Données d'équipe
On utilise les données de joueurs pour faire la liste des equipes, et on complete avec les divisions et conférence, puis on transfere

In [ ]:
dicoDivision={'Central Division': ['MIL', 'CHI', 'IND', 'CLE', 'DET'],'Atlantic Division': ['BOS', 'BKN', 'NYK', 'TOR', 
'PHI'],'Southeast Division' : ['MIA', 'ORL', 'WAS', 'CHA', 'ATL'],'Southwest Division': ['SAS', 'HOU', 'DAL', 'NOP', 
'MEM'],'Northwest Division' : ['DEN', 'MIN', 'OKC', 'POR', 'UTA'],'Pacific Division' : ['LAC', 'LAL', 'PHX', 'GSW', 'SAC']}
listDivision=[k for e in pagejoueuer.dfJoueurs.equipe.unique() for k,v in dicoDivision.items()  if e in v] 

In [ ]:
dicoConference={'Ouest' : ['SAS', 'HOU', 'DAL', 'NOP', 'MEM','DEN', 'MIN', 'OKC', 'POR', 'UTA','LAC', 'LAL', 'PHX', 
    'GSW', 'SAC'],'Est':['MIL', 'CHI', 'IND', 'CLE', 'DET','BOS', 'BKN', 'NYK', 'TOR', 'PHI','MIA', 
                         'ORL', 'WAS', 'CHA', 'ATL']}
dicoNom={'MIA':'Miami Heat','MIL':'Milwaukee Bucks','NOP':'New-Orleans Pelicans','SAS':'San-Antonio Spurs',
'PHX':'Phoenix','MEM':'Memphis Grizzlies',    'BKN':'Brooklyn Nets','ORL':'Orlando Magic','LAL':'LosAngeles Lakers','POR':'Portland TrailBlazzers','TOR':'Toronto Raptors','CHI':'Chicago Bulls',
'OKC':'Oklahoma City Thunder','WAS':'Washington Wizards','UTA':'Utah Jazz','SAC':'Sacramento Kings','CHA':'Charlotte Hornets','NYK':'New-York Knicks','DEN':'Denver Nuggets',
'LAC':'Los Angeles Clippers','GSW':'Golden State Warriors','MIN':'Minessota TimberWolves','DET':'Detroit Pistons','DAL':'Dallas Mavericks',
'IND':'Indiana Pacers','ATL':'Atlanta Hawks','CLE':'Cleveland Cavaliers','PHI':'Philadelphia Sixers','BOS':'Boston Celtics','HOU':'Houston Rockets'}
listConference=[k for e in pagejoueuer.dfJoueurs.equipe.unique() for k,v in dicoConference.items()  if e in v] 
listNom=[v for k,v in dicoNom.items() for e in pagejoueuer.dfJoueurs.equipe.unique() if k==e] 

In [ ]:
with ct.ConnexionBdd('basket','boulot') as c : 
    pd.DataFrame({'id_equipe':pagejoueuer.dfJoueurs.equipe.unique(),'nom_equipe': listNom,
              'conference':listConference, 'division':listDivision}).to_sql(
        'equipe', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)

# 3. Données de match
Dans un premier temp on va lire les fichiers csv stcokés [ici](C:\Users\martin.schoreisz\Documents\AffairesEnCours\temp\basket)
L'idée c'est d'avoir une classe match, qui va regrouper : 
- les df de matchs, statjoueur, stat
- va falloir trier les joueuers qui n'ont pas joué avant de les basculer dans les stats joueur (attribut dnp)
- va falloir trouver les joueurs non contenus dans la base des joueurs et les ajouter (nom à minima)
- va falloir gérer les blessures  : 
> - les ajouter à la table blessure si elle ne sont pas déjà dedans
> - noter comme apte les joueurs présents dans la table blessure, dont la date de guérison n'est pas connue

In [ ]:
dossierBase=r'C:\Users\martin.schoreisz\git\Basket\Basket\data'

In [ ]:
with ct.ConnexionBdd('basket','maison') as c:
    for root, dossier, files in os.walk(dossierBase) : 
        for f in files :
            id_type_match=0
            id_saison=1
            if 'equipe' in f and f.endswith('.csv'): 
                continue
            elif 'equipe' not in f and f.endswith('.csv'):
                #recuperer l'id_match : 
                idMatch=c.sqlAlchemyConn.execute("SELECT last_value FROM donnees_source.match_id_match_seq").fetchone()[0] 
                #recuperer l'identifiant du match dans le dossier contenant les fichiers
                groupe=f.split('_')[0][1:]
                #recuperer la date du match
                date=f.split('_')[1][:-4]
                #lire le fichier csv du match, transformer pour remplir la table des matchs
                dfMatchCsv=pd.read_csv(os.path.join(root,f)).rename(columns={'equipe':'id_equipe'})
                equipeExt=dfMatchCsv.iloc[0].id_equipe
                equipeDom=dfMatchCsv.iloc[1].id_equipe
                dfMatchBdd=pd.DataFrame({'id_saison':[id_saison],
                                      'date_match':[date], 
                                      'equipe_domicile':[equipeDom], 
                                      'equipe_exterieure':[equipeExt],
                                      'id_type_match':[id_type_match],
                                        'id_match':idMatch})
                #melt la table csv pour remplir la table des scores de match
                dfScoreMatch=dfMatchCsv.melt(id_vars=['id_equipe'], value_vars=['q1','q2','q3','q4','final'],
                            var_name='id_periode', value_name='score_periode').sort_values('id_equipe')
                dfScoreMatch['id_match']=idMatch
                
                #maintenant il faut lire les fichiers stats_equipe associés
                for i in (0,1) :
                    nomFichierEquipe=f'm{groupe}_equipe{i}_{date}.csv'
                    dfStatsJoueursCsv=pd.read_csv(os.path.join(root,nomFichierEquipe))
                    #le pb ça va etre de recuperer les id_joueurs, va falloir telecharger d'abords la table de correspondance
                    #faire la jointure en amont et ensuite on bascule tout le monde
                    dfJoueursBdd=pd.read_sql('select * from donnees_source.joueur',c.sqlAlchemyConn)
                    dfJoueursTot=dfStatsJoueursCsv.merge(dfJoueursBdd, on='nom')

                    #pensez à checker si tous les joueurs sont présents dans la base joueur, et les ajouter si ils n'y sont pas
                    dfJoueurInconnus=dfStatsJoueursCsv.loc[~dfStatsJoueursCsv.nom.isin(dfJoueursBdd.nom.tolist())]
                    if not dfJoueurInconnus.empty : 
                        dfJoueurInconnus=dfJoueurInconnus[['nom', 'position']]
                        #basculer dans Bdd
                        #dfJoueurInconnus.to_sql('joueur', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)
                        print(f"Joueurs inconnus : {','.join(dfJoueurInconnus.nom.tolist())}")

                    #refaire un import et jointure au cas où des joueurs aient été ajoutés
                    dfJoueursBddFinal=pd.read_sql('select * from donnees_source.joueur',c.sqlAlchemyConn)
                    dfJoueursTot=dfStatsJoueursCsv.merge(dfJoueursBddFinal, on='nom')

                    #checker si des joueurs sont notés blessé et ne sont pas déjà dans la base blessure
                    dfJoueursBlesses=dfStatsJoueursCsv.loc[dfStatsJoueursCsv['blesse']]
                    rqtBlessesEncours="""select b.*,j.nom 
                                          from donnees_source.joueur j JOIN donnees_source.blessure b ON j.id_joueur=b.id_joueur 
                                          WHERE b.date_guerison IS NULL"""
                    dfJoueursBlessesBdd=pd.read_sql(rqtBlessesEncours,c.sqlAlchemyConn)
                    if not dfJoueursBlesses.empty : #si des joueurs blesses, verifier qu'ils ne sont pas déjà presents dans la base
                        if dfJoueursBlesses.loc[~dfJoueursBlesses.nom.isin(dfJoueursBlessesBdd)] : 
                            dfJoueursBlesses.merge(dfJoueursBddFinal, on='nom')#pour récupérer l'id_joueur
                            print(','.join(dfJoueursBlesses.nom.tolist()))
                            #dfJoueursBlesses[['id_joueur']].assign(date_blessure=date).to_sql('blessure', c.sqlAlchemyConn, schema='donnees_source', if_exists='append', index=False)
                    #checker si des joueurs ayant joué étaient noté blessé mais sont revenus


                    #pensez à enlever du fichier source les joueurs n'ayant pas joué avant de creer la df des statsJoueur: 
                    dfStatsJoueursActifCsv=dfStatsJoueursCsv.loc[~dfStatsJoueursCsv['dnp']]

                    #ensuite mettre en forme les stats de joueurs
                    dfStatsJoueursBdd=dfJoueursTot[['minute', 'points', 'rebonds', 'passes_dec', 'steal',
                       'contres', 'tir_reussi', 'tir_tentes', 'pct_tir', 'trois_pt_r',
                       'trois_pt_t', 'pct_3_pt', 'lanc_frc_r', 'lanc_frc_t', 'pct_lfrc',
                       'rebonds_o', 'rebonds_d', 'ball_perdu', 'faute_p', 'plus_moins', 'dnp',
                       'blesse', 'score_ttfl', 'id_joueur']].copy()
                    dfStatsJoueursBdd['id_match']=idMatch
        else :
            continue

In [ ]:
dfJoueursBlessesBdd

In [ ]:
dfJoueurInconnus=dfStatsJoueursActifCsv.loc[~dfStatsJoueursActifCsv.nom.isin(dfJoueursBdd.nom.tolist())]

In [ ]:
dfJoueurInconnus

# 4. exemple de blessure
le joueur Markelle Fultz s'est fait une rupture des ligaments croisé, exemple d'insertion dans la table blessure

In [ ]:
nomJoueurBlesse='Markelle Fultz'
idTypeBlessure=1
dateBlessure='2021-06-01'
requeteBlessure = f"""insert into donnees_source.blessure(id_joueur,id_type_blessure,date_blessure) 
             select id_joueur,{idTypeBlessure},{dateBlessure} from donnees_source.joueur where lower(nom)=lower{nomJoueurBlesse}"""
with ct.ConnexionBdd('basket','maison') as c : 
    c.execute(c.sqlAlchemyConn,requeteBlessure)

# 5. exemple de contrat
[page nba](https://www.nba.com/players/transactions)